In [1]:
import recwizard
import os
from recwizard import FillBlankConfig, FillBlankPipeline
from recwizard import ExpansionConfig, ExpansionPipeline
from recwizard import UnicrsRec, UnicrsGen
from recwizard import ChatgptGen, RedialRec

c:\Users\USER\Desktop\24_winter\RecSys_and_LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\USER\Desktop\24_winter\RecSys_and_LLM\.venv\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\USER\Desktop\24_winter\RecSys_and_LLM\.venv\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
##################
# Recommendation #
##################
unicrs_rec = UnicrsRec.from_pretrained('recwizard/unicrs-rec-redial')
##################
#   Generation   #
##################
unicrs_gen = UnicrsGen.from_pretrained('recwizard/unicrs-gen-redial')
gpt_gen = ChatgptGen.from_pretrained('RecWizard/chatgpt-gen-fillblank')
##################
# Configurations #
##################
fill_blank_config = FillBlankConfig()
expansion_config = ExpansionConfig()

c:\Users\USER\Desktop\24_winter\RecSys_and_LLM\.venv\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\USER\Desktop\24_winter\RecSys_and_LLM\.venv\Lib\site-packages\transformers\modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless th

In [3]:
def set_pipeline(flag, rec_module, gen_module):
    if flag.lower() == 'blank':
        pipeline = FillBlankPipeline(
            config=fill_blank_config,
            rec_module=rec_module,
            gen_module=gen_module
        )
    elif flag.lower() == 'expansion':
        pipeline = ExpansionPipeline(
            config=expansion_config,
            rec_module=rec_module,
            gen_module=gen_module
    )
    else:
        pipeline = FillBlankPipeline(
            config=fill_blank_config,
            rec_module=unicrs_rec,
            gen_module=gen_module
        )
    return pipeline

In [4]:
def chat_with_system(pipeline):
    context = []
    print("대화를 시작합니다. 종료하려면 'quit'를 입력하세요.")

    while True:
        user_input = input("User: ")
        if user_input.lower() == 'quit':
            break

        # 사용자 입력을 컨텍스트에 추가
        context.append(f"User: {user_input}")

        # 전체 컨텍스트를 <sep>로 연결
        full_context = "<sep>".join(context)

        # 응답 생성
        response = pipeline.response(full_context)
        print("System:", response)

        # 시스템 응답을 컨텍스트에 추가
        context.append(f"System: {response}")

In [5]:
dialog = [
    "User: Hello!",
    "System: Hello, I have some movie ideas for you. Have you watched the movie <entity>Interstella</entity>?",
    "User: Yes, i've seen that movie. And i'm Looking for other movies in the horor category."
]

In [6]:
def test_single_turn(pipeline, dialog):
    # 대화 컨텍스트 설정
    context = "<sep>".join(dialog)
    # print(context)
    answer = pipeline.response(context)
    print(answer)
    dialog.append(f"System: {answer}")

In [7]:
pipeline = set_pipeline('gpt', unicrs_rec, gpt_gen)
test_single_turn(pipeline, dialog)
#chat_with_system(pipeline)

System: Sure, if you're into horror, you might want to check out It (2017). It's really popular in the genre.


In [8]:
# dialog에 사용자의 입력을 추가하는 메서드
def add_user_input(dialog, user_input):
    dialog.append(f"User: {user_input}")

In [9]:
add_user_input(dialog, "thank you, i will check that out. Can you recommend me a movie that is similar to the movie <entity>Interstella</entity>?")
test_single_turn(pipeline, dialog)

System: Sure, if you enjoyed Interstellar, you might also like Interstellar (2014). It has a similar theme of space exploration and a complex storyline.
